In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.tz import tzlocal

# caiman
import caiman as cm
from caiman.paths import caiman_datadir
from caiman.utils.utils import download_demo
from caiman.source_extraction.cnmf import params as params
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf

In [ ]:
caiman_datadir()

In [ ]:
fnames = [join_file_path(
    caiman_datadir(), 
    'example_movies',
    'Sue_2x_3000_40_-46.nwb'
)]
# estimates save path can be same or different from raw data path

save_path = join_file_path(
    caiman_datadir(), 
    'example_movies',
    'Sue_2x_3000_40_-46_CNMF_estimates.nwb'
)
# filename to be created or processed

# dataset dependent parameters
fr = 15.  # imaging rate in frames per second
decay_time = 0.4  # length of a typical transient in seconds

### CaImAnの関数でNWB保存

In [ ]:
# NWBがない場合保存する
if not os.path.exists(fnames[0]):
    
    # tifロードする
    fnames_orig = 'Sue_2x_3000_40_-46.tif'
    if fnames_orig in ['Sue_2x_3000_40_-46.tif', 'demoMovie.tif']:
        fnames_orig = [download_demo(fnames_orig)]
    orig_movie = cm.load(fnames_orig, fr=fr)

    # save file in NWB format with various additional info
    orig_movie.save(
        fnames[0], 
        sess_desc='test', 
        identifier='demo 1',
        imaging_plane_description='single plane',
        emission_lambda=520.0, indicator='GCAMP6f',
        location='parietal cortex',
        experimenter='Sue Ann Koay', lab_name='Tank Lab',
        institution='Princeton U',
        experiment_description='Experiment Description',
        session_id='Session 1',
        var_name_hdf5='TwoPhotonSeries')

### Motion Correctのパラメータ

In [ ]:
# motion_correction parameter
dxy = (2., 2.)  # spatial resolution in x and y in (um per pixel)
# note the lower than usual spatial resolution here
max_shift_um = (12., 12.)  # maximum shift in um
patch_motion_um = (100., 100.)  # patch size for non-rigid correction in um
pw_rigid = True       # flag to select rigid vs pw_rigid motion correction
# maximum allowed rigid shift in pixels
max_shifts = [int(a/b) for a, b in zip(max_shift_um, dxy)]
# start a new patch for pw-rigid motion correction every x pixels
strides = tuple([int(a/b) for a, b in zip(patch_motion_um, dxy)])
# overlap between patches (size of patch in pixels: strides+overlaps)
overlaps = (24, 24)
# maximum deviation allowed for patch with respect to rigid shifts
max_deviation_rigid = 3

mc_dict = {
    'fnames': fnames,
    'fr': fr,
    'decay_time': decay_time,
    'dxy': dxy,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': 'copy',
    #'var_name_hdf5': 'acquisition/TwoPhotonSeries'
    'var_name_hdf5': 'TwoPhotonSeries'
}

opts = params.CNMFParams(params_dict=mc_dict)

### motion correction

In [ ]:
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
param = opts.get_group('motion')
# param['pw_rigid'] = False

mc = MotionCorrect(
    fnames, 
    dview=dview, 
    var_name_hdf5=opts.data['var_name_hdf5'], 
    **param
)
mc.motion_correct(save_movie=True)

In [ ]:
# %% MEMORY MAPPING
border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0
# you can include the boundaries of the FOV if you used the 'copy' option
# during motion correction, although be careful about the components near
# the boundaries

# memory map the file in order 'C'
fname_new = cm.save_memmap(
    mc.mmap_file, base_name='memmap_', order='C',
    border_to_0=border_to_0)  # exclude borders

In [ ]:
# now load the file
Yr, dims, T = cm.load_memmap(fname_new)   # (3000, 28900)
mc_images = np.reshape(Yr.T, [T] + list(dims), order='F')

### CNMF

In [ ]:
# parameter
p = 1                    # order of the autoregressive system
gnb = 2                  # number of global background components
merge_thr = 0.85         # merging threshold, max correlation allowed
rf = 15
# half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 6          # amount of overlap between the patches in pixels
K = 4                    # number of components per patch
gSig = [4, 4]            # expected half size of neurons in pixels
# initialization method (if analyzing dendritic data using 'sparse_nmf')
method_init = 'greedy_roi'
ssub = 2                     # spatial subsampling during initialization
tsub = 2                     # temporal subsampling during intialization

# parameters for component evaluation
opts_dict = {
    'fnames': fnames,
     'fr': fr,
     'nb': gnb,
     'rf': rf,
     'K': K,
     'gSig': gSig,
     'stride': stride_cnmf,
     'method_init': method_init,
     'rolling_sum': True,
     'merge_thr': merge_thr,
     'n_processes': n_processes,
     'only_init': True,
     'ssub': ssub,
     'tsub': tsub
}

opts.change_params(params_dict=opts_dict);
# %% RUN CNMF ON PATCHES
# First extract spatial and temporal components on patches and combine them
# for this step deconvolution is turned off (p=0)
opts.change_params({'p': 0})

In [ ]:
# fp = np.memmap('a.mmap', dtype='float32', mode='w+', shape=images.shape, order='C')
# fp[:] = images[:]

In [ ]:
cm.mmapping.load_memmap(fname_new)

In [ ]:
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit(mc_images)

In [ ]:
# %% plot contours of found components
Cn = cm.local_correlations(mc_images, swap_dim=False)
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours(img=Cn)
plt.title('Contour plots of found components')

#%% save results in a separate file (just for demonstration purposes)
cnm.estimates.Cn = Cn
# cnm.save(fname_new[:-4]+'hdf5')
#cm.movie(Cn).save(fname_new[:-5]+'_Cn.tif'

In [ ]:
print(cnm.estimates.C.shape)
print(cnm.estimates.f.shape)

In [ ]:
cnm.estimates.detrend_df_f()

In [ ]:
cnm.estimates.F_dff.shape

In [ ]:
cnm.estimates.evaluate_components(fp, cnm.params, dview=dview)

In [ ]:
cnm.estimates.save_NWB(
    save_path, 
    imaging_rate=fr, 
    session_start_time=datetime.now(tzlocal()), 
    raw_data_file=fnames[0]
)

In [ ]:
fr

### NWB保存

In [ ]:
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.base import Images
from pynwb.image import GrayscaleImage
from pynwb.device import Device
from pynwb.ophys import (
    OpticalChannel, ImageSeries, ImageSegmentation,
    Fluorescence, MotionCorrection, CorrectedImageStack
)

In [ ]:
# NWBFile作成
identifier = 'CaImAn'
session_start_time=datetime.now(tzlocal())
exp_desc=None

nwbfile = NWBFile(
    'CaImAn Results', 
    identifier, 
    session_start_time, 
    experiment_description=exp_desc
)

In [ ]:
# deviceを追加
device = Device('imaging_device')
nwbfile.add_device(device)

In [ ]:
# aquisitionを追加
emission_lambda=520.0
excitation_lambda=488.0
imaging_rate=30.
indicator='OGB-1'
location='brain'

optical_channel = OpticalChannel(
    'OpticalChannel',
    'main optical channel',
    emission_lambda=emission_lambda
)

nwbfile.create_imaging_plane(
    name='ImagingPlane',
    optical_channel=optical_channel,
    description='some imaging plane description',
    device=device,
    excitation_lambda=excitation_lambda,
    imaging_rate=imaging_rate,
    indicator=indicator,
    location=location
)

nwbfile.add_acquisition(
    ImageSeries(
        name='TwoPhotonSeries',
        external_file=[fnames[0]],
        format='external',
        rate=imaging_rate,
        starting_frame=[0]
))

In [ ]:
nwbfile.processing.values()

In [ ]:
nwbfile.imaging_planes['ImagingPlane'].imaging_rate

### ophys

In [ ]:
nwbfile.create_processing_module(
    'ophys', 'contains caiman estimates for the main imaging plane')
# ophys_module = nwbfile.create_processing_module(
#     'ophys', 'contains caiman estimates for the main imaging plane')

In [ ]:
nwbfile.processing['ophys']

In [ ]:
border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0

# # memory mapping
# fname_new = save_memmap(
#     mc.mmap_file,
#     base_name='memmap_',
#     order='C',
#     border_to_0=border_to_0
# )

# # now load the file
# Yr, dims, T = load_memmap(fname_new)
# images = np.array(np.reshape(
#     Yr.T, [T] + list(dims), order='F'))
# mc_images = Yr.T.reshape((T,) + dims, order='F')
# mc_images = np.array(mc_images)

In [ ]:
fnames_orig = 'Sue_2x_3000_40_-46.tif'
if fnames_orig in ['Sue_2x_3000_40_-46.tif']:
    fnames_orig = [download_demo(fnames_orig)]
orig_movie = cm.load(fnames_orig, fr=fr)

### motion correction

In [ ]:
# motion correction
original = ImageSeries(
    name='original',  # this must be named "corrected"
    data=orig_movie,
    unit='na',
    format='raw',
    starting_time=0.0,
    rate=1.0
)

corrected = ImageSeries(
    name='corrected',  # this must be named "corrected"
    data=mc_images,
    unit='na',
    format='raw',
    starting_time=0.0,
    rate=1.0
)

xy_translation_data =  (np.array(mc.x_shifts_els), np.array(mc.y_shifts_els)) if param['pw_rigid'] else np.array(mc.shifts_rig)
xy_translation = TimeSeries(
    name='xy_translation',
    data=xy_translation_data,
    unit='pixels',
    starting_time=0.0,
    rate=1.0,
)

corrected_image_stack = CorrectedImageStack(
    corrected=corrected,
    original=original,
    xy_translation=xy_translation,
)

motion_correction = MotionCorrection(
    corrected_image_stacks=corrected_image_stack
)
nwbfile.processing['ophys'].add(motion_correction)

In [ ]:
nwbfile.processing['ophys'].add(ImageSegmentation())
# nwbfile.processing['ophys'].add_data_interface(fl)

In [ ]:
# with NWBHDF5IO('aa.nwb', 'w') as io:
#     io.write(nwbfile)

In [ ]:
# plane_segmentation = nwbfile.processing['ophys'].data_interfaces[
#     'ImageSegmentation'].create_plane_segmentation(
#     name='PlaneSegmentation',
#     description='CNMF_ROIs',
#     imaging_plane=list(nwbfile.imaging_planes.values())[0],
#     reference_images=list(nwbfile.acquisition.values())[0]
# )

In [ ]:
nwbfile.processing['ophys'].data_interfaces['ImageSegmentation'].create_plane_segmentation(
    name='PlaneSegmentation',
    description='CNMF_ROIs',
    imaging_plane=list(nwbfile.imaging_planes.values())[0],
    reference_images=list(nwbfile.acquisition.values())[0]
)
plane_segmentation = nwbfile.processing['ophys'].data_interfaces[
        'ImageSegmentation'].plane_segmentations['PlaneSegmentation']

# plane_segmentation.add_column('r', 'description of r values')
# plane_segmentation.add_column('snr', 'signal to noise ratio')

plane_segmentation.add_column('accepted', 'in accepted list')
plane_segmentation.add_column('rejected', 'in rejected list')

if cnm.estimates.cnn_preds is not None:
    plane_segmentation.add_column('cnn', 'description of CNN')
if cnm.estimates.idx_components is not None:
    plane_segmentation.add_column('keep', 'in idx_components')

In [ ]:
'accepted' in plane_segmentation.colnames

In [ ]:
# plane_segmentation = nwbfile.processing['ophys'].data_interfaces[
#         'ImageSegmentation'].create_plane_segmentation(
#     name='PlaneSegmentation',
#     description='CNMF_ROIs',
#     imaging_plane=list(nwbfile.imaging_planes.values())[0],
#     reference_images=list(nwbfile.acquisition.values())[0]
# )

# # plane_segmentation.add_column('r', 'description of r values')
# # plane_segmentation.add_column('snr', 'signal to noise ratio')

# plane_segmentation.add_column('accepted', 'in accepted list')
# plane_segmentation.add_column('rejected', 'in rejected list')

# if cnm.estimates.cnn_preds is not None:
#     plane_segmentation.add_column('cnn', 'description of CNN')
# if cnm.estimates.idx_components is not None:
#     plane_segmentation.add_column('keep', 'in idx_components')

### ROIのカラムを追加

In [ ]:
n_cells = cnm.estimates.A.shape[-1]
for i in range(n_cells):
    add_roi_kwargs = dict(
        image_mask=cnm.estimates.A.T[i].T.toarray().reshape(cnm.estimates.dims),
#         r=cnm.estimates.r_values[i],
#         snr=cnm.estimates.SNR_comp[i],
        accepted=False,
        rejected=False
    )

    if hasattr(cnm.estimates, 'accepted_list'):
        add_roi_kwargs.update(accepted=i in cnm.estimates.accepted_list)
    if hasattr(cnm.estimates, 'rejected_list'):
        add_roi_kwargs.update(rejected=i in cnm.estimates.rejected_list)
    if cnm.estimates.cnn_preds is not None:
        add_roi_kwargs.update(cnn=cnm.estimates.cnn_preds[i])
    if cnm.estimates.idx_components is not None:
        add_roi_kwargs.update(keep=i in cnm.estimates.idx_components)

    plane_segmentation.add_roi(**add_roi_kwargs)

### backgroundsのカラムを追加

In [ ]:
# Backgrounds
for bg in cnm.estimates.b.T:
    add_bg_roi_kwargs = dict(
        image_mask=bg.reshape(cnm.estimates.dims),
#         r=np.nan,
#         snr=np.nan,
        accepted=False,
        rejected=False
    )
    if 'keep' in plane_segmentation.colnames:
        add_bg_roi_kwargs.update(keep=False)
    if 'cnn' in plane_segmentation.colnames:
        add_bg_roi_kwargs.update(cnn=np.nan)
    plane_segmentation.add_roi(**add_bg_roi_kwargs)

In [ ]:
add_roi_kwargs

In [ ]:
import caiman.utils.visualization as visualization
cont = visualization.get_contours(
    cnm.estimates.A, cnm.dims, thr=0.9, thr_method='nrg', swap_dim=False)
cont_cent = np.zeros([len(cont), 2])
for i in range(len(cont)):
    cont_cent[i, :] = np.nanmean(cont[i]['coordinates'], axis=0)
iscell = np.zeros(cont_cent.shape[0])
iscell[cnm.estimates.idx_components] = 1

In [ ]:
add_bg_roi_kwargs["image_mask"].shape

In [ ]:
iscell.reshape(1,)

In [ ]:
plane_segmentation.add_roi("iscell", "", iscell.reshape(-1, 1))

### Fluorescenceを登録

In [ ]:
fluo = Fluorescence()
nwbfile.processing['ophys'].add(fluo)

### FluorescenceにROItableを追加

In [ ]:
starting_time = 0
timestamps = np.arange(cnm.estimates.f.shape[1]) / imaging_rate + starting_time
n_rois = cnm.estimates.A.shape[-1]
n_bg = len(cnm.estimates.f)

# Neurons

# ROI
#### ROI tableを作成
data_interfaces = nwbfile.processing['ophys'].data_interfaces
plane_seg = data_interfaces['ImageSegmentation'].plane_segmentations['PlaneSegmentation']
fluo = data_interfaces['Fluorescence']

In [ ]:
# add roi
table_name = 'ROIs'
region = list(range(n_rois))
name = 'RoiResponseSeries'
data = cnm.estimates.C.T
unit = 'lumens'

region_roi = plane_seg.create_roi_table_region(
    table_name, region=region)

fluo.create_roi_response_series(
    name=name,
    data=data,
    rois=region_roi,
    unit=unit,
    timestamps=timestamps
)

In [ ]:
# Background

table_name = 'Background'
region = list(range(n_rois, n_rois+n_bg))
name = 'Background_Fluorescence_Response'
data = cnm.estimates.f.T
unit = 'lumens'

rt_region_bg = plane_seg.create_roi_table_region(
    table_name, region=region)

fluo.create_roi_response_series(
    name=name,
    data=data,
    rois=rt_region_bg,
    unit='lumens',
    timestamps=timestamps
)

In [ ]:
nwbfile.processing['ophys'].add(TimeSeries(
    name='residuals',
    description='residuals',
    data=cnm.estimates.YrA.T,
    timestamps=timestamps, unit='NA'
))

In [ ]:
# if hasattr(cnm.estimates, 'Cn'):
#     images = Images('summary_images')
#     images.add_image(GrayscaleImage(name='local_correlations', data=cnm.estimates.Cn))

### NWB保存

In [ ]:
with NWBHDF5IO(save_path, 'w') as io:
    io.write(nwbfile)

In [ ]:
with NWBHDF5IO(save_path, 'r') as io:
    nwbfile = io.read()

In [ ]:
import h5py
def PrintOnlyDataset(name, obj):
    if isinstance(obj, h5py.Dataset):
        print(name)

with h5py.File(save_path, "r") as f:
    f.visititems(PrintOnlyDataset)

### cnm2

In [ ]:
cnm2.estimates.plot_contours(img=Cn, idx=cnm.estimates.idx_components)

In [ ]:
cnm.params.change_params({'p': p})
cnm2 = cnm.refit(images, dview=dview)

In [ ]:
# %% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 2  # signal to noise ratio for accepting a component
rval_thr = 0.85  # space correlation threshold for accepting a component
cnn_thr = 0.99  # threshold for CNN based classifier
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected

cnm2.params.set(
    'quality', {
        'decay_time': decay_time, 
        'min_SNR': min_SNR,
        'rval_thr': rval_thr,
        'use_cnn': True,
        'min_cnn_thr': cnn_thr,
        'cnn_lowest': cnn_lowest
    }
);
cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview)
#%%
cnm2.estimates.Cn = Cn
# cnm2.save(fname_new[:-4] + 'hdf5')
# %% PLOT COMPONENTS
cnm2.estimates.plot_contours(img=Cn, idx=cnm2.estimates.idx_components)

In [ ]:
#%% Extract DF/F values
cnm2.estimates.detrend_df_f(quantileMin=8, frames_window=250)

In [ ]:
#%% Show final traces
cnm2.estimates.view_components(img=Cn)

In [ ]:
cnm2.estimates.save_NWB(
    save_path, 
    imaging_rate=fr, 
    session_start_time=datetime.now(tzlocal()), 
    raw_data_file=fnames[0]
)